<a href="https://colab.research.google.com/github/Ashish-Soni08/100-Days-Of-Code/blob/main/Day_42_basic_hybrid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic AND Keyword Search (Hybrid Search)

We will take a look at how to use Pinecone to perform a semantic search, while applying a traditional keyword search.

In [ ]:
all_sentences = [
    "purple is the best city in the forest",
    "No way chimps go bananas for snacks!",
    "it is not often you find soggy bananas on the street",
    "green should have smelled more tranquil but somehow it just tasted rotten",
    "joyce enjoyed eating pancakes with ketchup",
    "throwing bananas on to the street is not art",
    "as the asteroid hurtled toward earth becky was upset her dentist appointment had been canceled",
    "I'm getting way too old. I don't even buy green bananas anymore.",
    "to get your way you must not bombard the road with yellow fruit",
    "Time flies like an arrow; fruit flies like a banana"
]

We will use the `sentence-transformers` library to build our sentence embeddings. It can be installed using `pip` like so:

In [ ]:
!pip install sentence-transformers sacremoses
!pip install pinecone-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.6 MB/s 
     |████████████████████████████████| 880 kB 62.4 MB/s 
     |████████████████████████████████| 4.7 MB 54.4 MB/s 
     |████████████████████████████████| 1.3 MB 53.5 MB/s 
     |████████████████████████████████| 120 kB 68.4 MB/s 
     |████████████████████████████████| 6.6 MB 61.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=20dff0856979a19b6dcf0350dd634b7fe98edaacbfaa2c1355cf5e0ca7ff3b3d
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=00b66369b45540e5ff738a7d7456c1beb00e1d0079b986a38758fd198c209c10
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9


*(The notebook may need to be restarted for the install to take effect)*

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base')

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/591 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

We use this pretrained sentence transformer model to encode the sentences.

In [ ]:
all_embeddings = model.encode(all_sentences)
all_embeddings.shape

(10, 768)

We have **10** embeddings, each with a dimensionality of *768*. For the keyword search we will also need to store our sentences. But for the keyword search to work we need *keywords*. So, we will use a 'word-level' tokenizer from the HuggingFace transformers library to break our text into words - for this we will use the [`transfo-xl-wt103` model](https://huggingface.co/transformers/model_doc/transformerxl.html). 

*(If needed, run `!pip install transformers` - although this package should have been install when installing `sentence-transformers` above)*

In [ ]:
from transformers import AutoTokenizer

# transfo-xl tokenizer uses word-level encodings
tokenizer = AutoTokenizer.from_pretrained('transfo-xl-wt103')

all_tokens = [tokenizer.tokenize(sentence.lower()) for sentence in all_sentences]
all_tokens[0]

['purple', 'is', 'the', 'best', 'city', 'in', 'the', 'forest']

We have everything we need, the dense vector representations of each sentence, and the stripped list of tokens for each sentence. So let's establish a connection to Pinecone ready for upserting our data.

Next we need to connect to a Pinecone instance, you can get a [free API key here](https://app.pinecone.io).

In [ ]:
import pinecone
pinecone.init(api_key='063315d4-bd99-478d-bd5c-7e5a3c7a7939', environment='us-west1-gcp')

We can check for existing indexes with:

In [ ]:
pinecone.list_indexes()

[]

There are none, so let's create a new index with `create_index` and connect with `Index`.

In [ ]:
pinecone.create_index(name='keyword-search', dimension=all_embeddings.shape[1])
index = pinecone.Index('keyword-search')

We now merge our data into a list of tuples, where each tuple is structured as `(id, value, metadata)`.

In [ ]:
upserts = []
for i, (embedding, tokens) in enumerate(zip(all_embeddings, all_tokens)):
    upserts.append((str(i), embedding.tolist(), {'tokens': tokens}))

In [ ]:
# then we upsert
index.upsert(vectors=upserts)

{'upserted_count': 10}

### Upsert with CURL

Alternatively, we can upsert using curl. For this we need to reformat our data and save it as a JSON file.

In [ ]:
import json

# reformat the data
upserts = {'vectors': []}
for i, (embedding, tokens) in enumerate(zip(all_embeddings, all_tokens)):
    vector = {'id':f'{i}',
              'values': embedding.tolist(),
              'metadata':{'tokens':tokens}}
    upserts['vectors'].append(vector)

# save to JSON
with open('./upsert.json', 'w') as f:
    json.dump(upserts, f, indent=4)

This produces a JSON containing a list of *10* dictionaries within the `vectors` key. Each dictionary contains the embeddings and metadata for a single sample in the format:

```json
{
    'id': 'sentence_n',
    'values': [0.001, 0.002, ...],
    'metadata': {
        'tokens': ['purple', 'is', ...]
    }
}
```

To upsert with curl, we first find the index URL in the [Pinecone dashboard](https://app.pinecone.io), for `https://keyword-search-1234.svc.us-west1-gcp.pinecone.io/vectors/upsert` so I'd type:

In [ ]:
!curl -X POST \
    https://keyword-search-1234.svc.us-west1-gcp.pinecone.io/vectors/upsert \
    -H 'Content-Type: application/json' \
    -H 'Api-Key: YOUR_API_KEY' \
    -d @./upsert.json

## Querying

We now have the data in our index, let's first perform a semantic search using a query sentence, we will return the most *semantically* similar sentences.

We define the query, and encode as we did for `all_sentences` before.

In [ ]:
query_sentence = "there is an art to getting your way and throwing bananas on to the street is not it"
xq = model.encode(query_sentence).tolist()

When querying with `index.query` we can pass the query vector as our first argument, and *later* when filtering for specific keywords we will add the `filter` parameter.

In [ ]:
result = index.query(xq, top_k=10, includeMetadata=True)
result

{'matches': [{'id': '5',
              'metadata': {'tokens': ['throwing',
                                      'bananas',
                                      'on',
                                      'to',
                                      'the',
                                      'street',
                                      'is',
                                      'not',
                                      'art']},
              'score': 0.732851863,
              'values': []},
             {'id': '8',
              'metadata': {'tokens': ['to',
                                      'get',
                                      'your',
                                      'way',
                                      'you',
                                      'must',
                                      'not',
                                      'bombard',
                                      'the',
                                      'road',
             

Let's extract just the sentence IDs to see the order of what we have returned.

In [ ]:
[x['id'] for x in result['matches']]

['5', '8', '2', '1', '9', '7', '0', '3', '4', '6']

Now let's add a keyword filter. Let's restrict the search to only return sentences that contain the word `bananas`.

In [ ]:
result = index.query(xq, top_k=10, filter={'tokens': 'bananas'})
[x['id'] for x in result['matches']]

['5', '2', '1', '7']

Again, let's extract IDs and then use these to see which sentences we're returning in the query above.

In [ ]:
ids = [int(x['id']) for x in result['matches']]
for i in ids:
    print(all_sentences[i])

throwing bananas on to the street is not art
it is not often you find soggy bananas on the street
No way chimps go bananas for snacks!
I'm getting way too old. I don't even buy green bananas anymore.


Okay cool, we can see that we're now filtering out all samples that do *not* contain the word 'bananas'. Maybe we'd like to extend this keyword filter further - for example we could filter for any samples that contain the word 'bananas' **OR** 'way' by modifying our filter to `{'$or': [{'tokens': 'bananas'}, {'tokens': 'way'}]}`.

In [ ]:
result = index.query(xq, top_k=10, filter={'$or': [
                         {'tokens': 'bananas'},
                         {'tokens': 'way'}
                     ]})

ids = [int(x['id']) for x in result['matches']]
for i in ids:
    print(all_sentences[i])

throwing bananas on to the street is not art
to get your way you must not bombard the road with yellow fruit
it is not often you find soggy bananas on the street
No way chimps go bananas for snacks!
I'm getting way too old. I don't even buy green bananas anymore.


Alternatively we can us the **in** `$in` condition rather than `$or` - it will produce the same results:

In [ ]:
result = index.query(xq, top_k=10, filter={
    'tokens': {'$in': ['bananas', 'way']}
})

ids = [int(x['id']) for x in result['matches']]
for i in ids:
    print(all_sentences[i])

throwing bananas on to the street is not art
to get your way you must not bombard the road with yellow fruit
it is not often you find soggy bananas on the street
No way chimps go bananas for snacks!
I'm getting way too old. I don't even buy green bananas anymore.


We could decide we only want to return samples that contain *both* 'bananas' **AND** 'way' by swapping the `$or` modifier for `$and`.

In [ ]:
result = index.query(xq, top_k=10, filter={'$and': [
                         {'tokens': 'bananas'},
                         {'tokens': 'way'}
                     ]})

ids = [int(x['id']) for x in result['matches']]
for i in ids:
    print(all_sentences[i])

No way chimps go bananas for snacks!
I'm getting way too old. I don't even buy green bananas anymore.


If we have a lot of keywords including every single one manually like above can quickly get tiresome, so we can just write something like this instead:

In [ ]:
keywords = ['bananas', 'way', 'green']
filter_dict = [{'tokens': word} for word in keywords]
filter_dict

[{'tokens': 'bananas'}, {'tokens': 'way'}, {'tokens': 'green'}]

And add it to our `query`.

In [ ]:
result = index.query(xq, top_k=10, filter={'$and': filter_dict})

ids = [int(x['id']) for x in result['matches']]
for i in ids:
    print(all_sentences[i])

I'm getting way too old. I don't even buy green bananas anymore.


We may also want to restrict our search to sentences that do *not* satisfy our conditions above, for example we may want all sentences that *do not* contain *'bananas'* but *do* contain *'way'*. To do this we can add **not equals** `$ne` to the `bananas` part of the query.

In [ ]:
result = index.query(xq, top_k=10, filter={'$and': [
                         {'tokens': {'$ne': 'bananas'}},
                         {'tokens': 'way'}
                     ]})

ids = [int(x['id']) for x in result['matches']]
for i in ids:
    print(all_sentences[i])

to get your way you must not bombard the road with yellow fruit


We can exclude multiple keywords too using the **not in** `$nin` condition.

In [ ]:
result = index.query(xq, top_k=10, filter={'tokens':
    {'$nin': ['bananas', 'way']}
})

ids = [int(x['id']) for x in result['matches']]
for i in ids:
    print(all_sentences[i])

Time flies like an arrow; fruit flies like a banana
purple is the best city in the forest
green should have smelled more tranquil but somehow it just tasted rotten
joyce enjoyed eating pancakes with ketchup
as the asteroid hurtled toward earth becky was upset her dentist appointment had been canceled
